In [ ]:
!pip install langchain chromadb sentence-transformers huggingface_hub transformers torch==2.6.0 torchvision==0.21.0 zipfile36 langchain-community langchain-huggingface -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.2 MB/s 

In [ ]:
import os
import zipfile
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_huggingface  import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from getpass import getpass

In [ ]:
# Set Hugging Face API key
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("Enter your Hugging Face API key: ")

Enter your Hugging Face API key: ··········


In [ ]:
# Download the ZIP file
url = "https://www.dropbox.com/scl/fi/p7d0zjnrefjne94j3ec2b/new_articles.zip?rlkey=izy5z44eofgx5fmjn8tif4y99&dl=1"
response = requests.get(url)
with open("/content/new_articles.zip", "wb") as f:
    f.write(response.content)

# Extract ZIP file
with zipfile.ZipFile("/content/new_articles.zip", "r") as zip_ref:
    zip_ref.extractall("/content/articles")

# Read and preprocess text files
articles = []
for file_name in os.listdir("/content/articles"):
    if file_name.endswith(".txt"):
        with open(f"/content/articles/{file_name}", "r", encoding="utf-8") as f:
            content = f.read().strip()
            articles.append({"content": content, "filename": file_name})

In [ ]:
articles[1]

{'content': "Generative AI is pretty impressive in terms of its fidelity these days, as viral memes like Balenciaga Pope would suggest. The latest systems can conjure up scenescapes from city skylines to cafes, creating images that appear startlingly realistic — at least on first glance.\n\nBut one of the longstanding weaknesses of text-to-image AI models is, ironically, text. Even the best models struggle to generate images with legible logos, much less text, calligraphy or fonts.\n\nBut that might change.\n\nLast week, DeepFloyd, a research group backed by Stability AI, unveiled DeepFloyd IF, a text-to-image model that can “smartly” integrate text into images. Trained on a dataset of more than a billion images and text, DeepFloyd IF, which requires a GPU with at least 16GB of RAM to run, can create an image from a prompt like “a teddy bear wearing a shirt that reads ‘Deep Floyd'” — optionally in a range of styles.\n\nDeepFloyd IF is available in open source, licensed in a way that pr

In [ ]:
# Chunk the articles
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = []
for article in articles:
    split_texts = splitter.split_text(article["content"])
    for i, text in enumerate(split_texts):
        chunks.append({
            "text": text,
            "metadata": {"filename": article["filename"], "chunk_id": i}
        })

In [ ]:
chunks[1]

{'text': 'Do you want This Week in Apps in your inbox every Saturday? Sign up here: techcrunch.com/newsletters\n\nTop Stories\n\nDorsey criticizes Twitter, Musk on the alternative social networks he’s backing\n\nAs demand for Bluesky, the Jack Dorsey-backed decentralized Twitter rival grows, the former Twitter CEO took to the app to share his thoughts on Twitter’s future, Elon Musk and the decision to take the company private. As TechCrunch’s Darrell Etherington reported, Dorsey responded to questions posed to him from other users and reporters on Bluesky, including one where he was asked if Musk has proven to be the best possible steward for the social network.\n\nDorsey said he had not:\n\nNo. Nor do I think he acted right after realizing his timing was bad. Nor do I think the board should have forced the sale. It all went south. But it happened and all we can do now is build something to avoid that ever happening again. So I’m happy Jay and team and nostr devs exist and building it.

In [ ]:
# Set up embeddings and ChromaDB
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_texts(
    texts=[chunk["text"] for chunk in chunks],
    embedding=embeddings,
    metadatas=[chunk["metadata"] for chunk in chunks],
    persist_directory="/content/chroma_db"
)

<ipython-input-8-3800308b58a8>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Set up FLAN-T5 model
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
print("Model loaded")
pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024,
    device= 0  # Use GPU
)
llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0


Model loaded


In [ ]:
# Create RetrievalQA chain
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
# Test the chatbot
query1 = "What is the main topic of the articles?"
print("Query 1:", query1)
result1 = qa_chain.invoke({"query": query1})
print("Answer 1:", result1["result"])

query2 = "What did Pando company do?"
print("Query 2:", query2)
result2 = qa_chain.invoke({"query": query2})
print("Answer 2:", result2["result"])

query3 = "Who did Databricks acquire?"
print("Query 3:", query3)
result3= qa_chain.invoke({"query": query3})
print("Answer 3:", result3["result"])

# print("\nSources:")
# for doc in result["source_documents"]:
#     print(f"- {doc.metadata['filename']} (Chunk {doc.metadata['chunk_id']})")

Query 1: What is the main topic of the articles?
Answer 1: AI
Query 2: What did Pando company do?
Answer 2: solve for global logistics through a software-as-a-service platform offering
Query 3: Who did Databricks acquire?
Answer 3: Okera
